In [54]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import math
import numpy as np
import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import torch.optim as optim


#### Transformer Encoder

In [84]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)
    

class Transformer(nn.Module):
    def __init__(self, d_model, heads, num_layers,dropout, max_len, output_classes):
        super().__init__()
        self.encoder_layer= nn.TransformerEncoderLayer(d_model=d_model, nhead=heads)
        self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.positional_encoder = PositionalEncoding(d_model=d_model, dropout=dropout, max_len=max_len)
        self.linear = nn.Linear(in_features=d_model, out_features=output_classes)
        
    def forward(self, x):
        print(x.shape)
        x1 = self.positional_encoder(x)
        x2 = self.encoder(x1)
        out = self.linear(x2)
        return out
        


kinematic_features = 22 #dmodel
heads = 11
num_layers = 6
dropout = 0.1
max_len = 100
output_classes = 7

# src = torch.rand(10, 10, kinematic_features)

model = Transformer(kinematic_features,heads,num_layers,dropout,max_len,output_classes)


#### Load Data

In [105]:
gesture_labels_directory = './new_dataset/peg_transfer/peg_transfer/labeled/gestures/'
window_size = 10 # 10 frame windows (kinematic data)
enite_dataset = []
counter = 0



class KinematicDataset(Dataset):
    
    def __init__(self,data_directory, window_size):
        
        self.directory = data_directory
        self.window_size = window_size
        self.le = preprocessing.LabelEncoder()
        self.enc = preprocessing.OneHotEncoder()
        (self.X, self.Y) = self._load_data()
        
    def _load_data(self):
        x = []
        y = []
        self.windows_per_file = []
        
        for filename in os.listdir(self.directory):
            f = os.path.join(self.directory, filename)
            # checking if it is a file
            if os.path.isfile(f):
                kinematics_data = pd.read_csv(f)
                kin_data = kinematics_data.iloc[:,:-1]
                kin_label = kinematics_data.iloc[:,-1]
                
                x.append(kin_data.values)
                y.append(kin_label.values)
                
                
                
                if len(self.windows_per_file) == 0:
                    self.windows_per_file = [len(kin_label) - self.window_size + 1]
                else:
                    self.windows_per_file.append(self.windows_per_file[-1] + len(kin_label) - self.window_size + 1)
        
        self.le.fit(y[0])
        y = [self.le.transform(yi) for yi in y]
        x = np.concatenate(x)
        y = np.concatenate(y)
        
        x = torch.from_numpy(x)
        y = torch.from_numpy(y)
        
        x = x.to(torch.float32)
        return (x, y)
    
    def __len__(self):
        # this should return the size of the dataset
        return self.Y.shape[0]
    def __getitem__(self, idx):
        # this should return one sample from the dataset
        features = self.X[idx : idx + window_size]
        target = self.Y[idx : idx + window_size]
        return features, target
                    
                    


In [127]:
# train_data, test_data = 

batch_size = 1

dataset = KinematicDataset(gesture_labels_directory,10)

train_set, test_set = torch.utils.data.random_split(dataset, [0.7, 0.3])

trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                          shuffle=True, num_workers=0)

testloader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
                                          shuffle=True, num_workers=0)

print(trainloader.batch_sampler)


In [125]:
# loss function
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [128]:
epochs = 10

for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        
        outputs = torch.transpose(outputs,1,2)
        
        print(outputs.shape,  labels.shape, inputs.shape)
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

torch.Size([32, 10, 22])
torch.Size([32, 7, 10]) torch.Size([32, 10]) torch.Size([32, 10, 22])
torch.Size([32, 10, 22])
torch.Size([32, 7, 10]) torch.Size([32, 10]) torch.Size([32, 10, 22])
torch.Size([32, 10, 22])
torch.Size([32, 7, 10]) torch.Size([32, 10]) torch.Size([32, 10, 22])
torch.Size([32, 10, 22])
torch.Size([32, 7, 10]) torch.Size([32, 10]) torch.Size([32, 10, 22])
torch.Size([32, 10, 22])
torch.Size([32, 7, 10]) torch.Size([32, 10]) torch.Size([32, 10, 22])
torch.Size([32, 10, 22])
torch.Size([32, 7, 10]) torch.Size([32, 10]) torch.Size([32, 10, 22])
torch.Size([32, 10, 22])
torch.Size([32, 7, 10]) torch.Size([32, 10]) torch.Size([32, 10, 22])
torch.Size([32, 10, 22])
torch.Size([32, 7, 10]) torch.Size([32, 10]) torch.Size([32, 10, 22])
torch.Size([32, 10, 22])
torch.Size([32, 7, 10]) torch.Size([32, 10]) torch.Size([32, 10, 22])
torch.Size([32, 10, 22])
torch.Size([32, 7, 10]) torch.Size([32, 10]) torch.Size([32, 10, 22])
torch.Size([32, 10, 22])
torch.Size([32, 7, 10]) t

RuntimeError: stack expects each tensor to be equal size, but got [10, 22] at entry 0 and [9, 22] at entry 18